In [1]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup as BS
import xmltodict

In [2]:
# Getting Parent URL
url="https://cloud.google.com/release-notes/all"
page=rq.get(url).content
soup=BS(page,'html.parser')
services = soup.findAll("li", {"class": "release-note-sitemap-card-item"})

In [3]:
root_url="https://cloud.google.com"
url_pack=[dict(name=i.find("h3").text,url=root_url+i.find("a")["href"]) for i in services]
df=pd.DataFrame(url_pack)
df.to_csv("release-pages.csv", index=False)

try:
    links=pd.read_csv("cache_url.csv")
    cache_url=list(links['url'])
except:
    cache_url=[]
df.head()

,name,url
0,AutoML Natural Language,https://cloud.google.com/automl-tables/docs/re...
1,AutoML Tables,https://cloud.google.com/natural-language/auto...
2,AutoML Translation,https://cloud.google.com/translate/automl/docs...
3,AutoML Video Intelligence Classification,https://cloud.google.com/video-intelligence/au...
4,AutoML Video Intelligence Object Tracking,https://cloud.google.com/video-intelligence/au...


In [4]:
def get_xml_links(col):
    if col in cache_url:
        get_url=links[links['url']==col]
        xml_url=list(get_url['xml_url'])[0]
    else:
        su=BS(rq.get(col).content,'html.parser')
        try:
            xml_url= su.find("section",{"class": "xml"}).find("code").text
            xml_url=xml_url.replace("\n","")
        except Exception as e:
            print("Error for ", col)
            xml_url="no_xml_link"
    return xml_url

In [5]:
df["xml_url"]=df.url.apply(get_xml_links)

In [6]:
df.to_csv("cache_url.csv",index=False)

In [7]:
xml_df=df[df["xml_url"]!="no_xml_link"]

In [8]:
def get_xml_data(row):
    print("working on link...  ",row)
    dataset=[]
    if row!="no_xml_link":
        try:
            docs=xmltodict.parse(rq.get(row).content)['feed']['entry']
            try:
                contents=docs["content"]
                c_soup=BS(contents["#text"],'html.parser')
                try:
                    status=c_soup.findAll("h3").text
                except:
                    try:
                        status=c_soup.find("h3").text
                    except:
                        status=''
                description=c_soup.text
                data= dict(release_date=docs['title'],content=contents, status=status, description=description)
                dataset.append(data)
            except:
                for i in docs:
                    try:
                        contents=i["content"]
                        c_soup=BS(contents["#text"],'html.parser')
                        try:
                            status=c_soup.findAll("h3").text
                        except:
                            try:
                                status=c_soup.find("h3").text
                            except:
                                status=''
                        description=c_soup.text
                        data= dict(release_date=i['title'],content=contents, status=status, description=description)
                        dataset.append(data)
                    except Exception as e:
                        print("Error ", e)            
        except Exception as e:
            print("xml parsing err", row)
    if len(dataset)==0:
        dataset="corrupted"
    return dataset


In [9]:
xml_df["datapack"]=df.xml_url.apply(get_xml_data)

working on link...   https://cloud.google.com/feeds/automl-tables-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/automl-language-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/automl-translate-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/automl-video-intelligence-classification-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/automl-video-intelligence-object-tracking-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/automl-vision-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/automl-vision-object-detection-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/vision-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/visionproductsearch-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/video-release-not

Entry pass
working on link...   https://cloud.google.com/feeds/gkeonprem-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/anthos-config-management-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/servicemesh-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/configconnector-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/runanthos-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/gcpmarketplace-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/migrateanthos-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/cloudbuild-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/trafficdirector-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/feeds/iot-release-notes.xml
Entry pass
working on link...   https://cloud.google.com/

/home/mahbub/anaconda3/envs/datascience/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
currupted_links=xml_df[xml_df["datapack"]=="corrupted"]
currupted_links

,name,url,xml_url,datapack
30,Cloud Healthcare API,https://cloud.google.com/healthcare/docs/relea...,https://cloud.google.com/feeds/cloud-healthcar...,corrupted
80,Cloud Tools for PowerShell,https://cloud.google.com/tools/powershell/docs...,https://cloud.google.com/feeds/cloud-tools-for...,corrupted
81,Cloud Tools for Visual Studio,https://cloud.google.com/tools/visual-studio/d...,https://cloud.google.com/feeds/cloud-tools-for...,corrupted
84,Cloud Healthcare API,https://cloud.google.com/healthcare/docs/relea...,https://cloud.google.com/feeds/cloud-healthcar...,corrupted


In [11]:
final_df=xml_df[xml_df["datapack"]!="corrupted"]
final_df.head()

,name,url,xml_url,datapack
0,AutoML Natural Language,https://cloud.google.com/automl-tables/docs/re...,https://cloud.google.com/feeds/automl-tables-r...,"[{'release_date': 'April 03, 2020', 'content':..."
1,AutoML Tables,https://cloud.google.com/natural-language/auto...,https://cloud.google.com/feeds/automl-language...,"[{'release_date': 'April 03, 2020', 'content':..."
2,AutoML Translation,https://cloud.google.com/translate/automl/docs...,https://cloud.google.com/feeds/automl-translat...,"[{'release_date': 'April 03, 2020', 'content':..."
3,AutoML Video Intelligence Classification,https://cloud.google.com/video-intelligence/au...,https://cloud.google.com/feeds/automl-video-in...,"[{'release_date': 'April 10, 2019', 'content':..."
4,AutoML Video Intelligence Object Tracking,https://cloud.google.com/video-intelligence/au...,https://cloud.google.com/feeds/automl-video-in...,"[{'release_date': 'September 23, 2019', 'conte..."


In [12]:
final_df.to_csv("need_to_flat.csv")